In [4]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graph

In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-20 14:31:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-03-20 14:31:05 (6.01 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [7]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData2").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [8]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t')

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [9]:
from pyspark.sql.functions import to_date

In [10]:
df.count()

5331449

In [12]:
# Drop NaN from dataset
df = df.dropna()

In [13]:
# Drop duplicates from dataset
df=df.distinct()

In [14]:
df.count()

5330701

In [15]:
review_id_table_1 = df.select(["review_id", "customer_id", "product_id","product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date")])
review_id_table_1.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1007YJJIFAC9J|   52769212|B009UHTGT0|     245155553| 2013-08-27|
|R100F8VKMH0GML|   22075855|B006Z96OI2|     556998710| 2015-02-28|
|R100MQXHRT2RWR|   49362036|B005PK7RW4|     475331940| 2015-06-07|
|R100Q7Y0SUHCK9|   22694784|B007UZNO4W|     240118513| 2014-08-10|
|R100QVXPVZS0DZ|   30117376|B00GZRPIEU|     206581276| 2015-01-14|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [16]:
products_1 = df.select(["product_id","product_title"]).distinct()
products_1.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00340NV90|New Massage Table...|
|B00N2BW2PK|Fitbit Charge HR ...|
|B00068JKNO|Swanson Coq10 100...|
|B001D8ZGAM|Aquasentials Easy...|
|B000PHFGV6|Tantus A Bomb Ana...|
+----------+--------------------+
only showing top 5 rows



In [17]:
customers_1 = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_1.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45641495|             6|
|   48343488|             1|
|   17171509|            18|
|    7255220|             2|
|   10262401|             2|
+-----------+--------------+
only showing top 5 rows



In [18]:
vine_table_1 = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_1.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1007YJJIFAC9J|          5|            0|          0|   N|
|R100F8VKMH0GML|          5|            0|          0|   N|
|R100MQXHRT2RWR|          1|            0|          0|   N|
|R100Q7Y0SUHCK9|          4|            0|          0|   N|
|R100QVXPVZS0DZ|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [19]:
 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cnnfcbfbmueb.us-west-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "qazWSXqazWSX",
          "driver": "org.postgresql.Driver"}

In [20]:
# Write review_id_df to table in RDS
review_id_table_1.write.jdbc(url=jdbc_url, table='review_id_table_1', mode=mode, properties=config)

In [21]:
# Write review_id_df to table in RDS
products_1.write.jdbc(url=jdbc_url, table='products_1', mode=mode, properties=config)

In [22]:
# Write review_id_df to table in RDS
customers_1.write.jdbc(url=jdbc_url, table='customers_1', mode=mode, properties=config)

In [23]:
# Write review_id_df to table in RDS
vine_table_1.write.jdbc(url=jdbc_url, table='vine_table_1', mode=mode, properties=config)